# DASK

In [ ]:
option_comp = 'local'

In [ ]:
from dask.distributed import Client

if option_comp == "merlin":

    from dask_jobqueue import SLURMCluster
    
    cluster = SLURMCluster(cores     = 8, 
                           memory    ='4GB', 
                           walltime  = '06:00:00',
                           interface ='ib0',
                           local_directory = '/data/user/kim_a',
                           log_directory   = '/data/user/kim_a',
                           job_extra="--exclusive") 
    
elif option_comp == "local":

    from dask.distributed import LocalCluster
    
    cluster = LocalCluster()

In [ ]:
client = Client(cluster)

In [ ]:
n_workers = 8
cluster.scale(n_workers)

In [ ]:
client

In [ ]:
# client.close()
# cluster.close()

# Conventional reference model

In [ ]:
from cge_klausen import parameters as cge_parameters
from cge_model import GeothermalConventionalModel
from setup_files_gsa import *

n_iter = 1000
project = 'Geothermal'
option = 'cge'
seed = 13413203

electricity_conv_prod, methods, cge_model, cge_parameters = setup_gt_project(project, option)

# Generate stochastic values
cge_parameters.stochastic(iterations=n_iter, seed=seed)
# Reference model
cge_parameters_sto = cge_model.run_ps(cge_parameters)

In [ ]:
%%time 
ref_cge = run_mc(cge_parameters_sto, electricity_conv_prod, methods, n_iter)

In [ ]:
ref_cge

# Computations with dask

In [ ]:
import dask

In [ ]:
def task_per_worker(n_workers, i_worker, option, n_iter, seed=seed):
    project = 'Geothermal'
    electricity_prod, methods, gt_model, parameters = setup_gt_project(project, option)
    # Generate stochastic values
    parameters.stochastic(iterations=n_iter, seed=seed) # TODO add seed, also as the function input
    # Reference model
    parameters_sto = gt_model.run_ps(parameters)
    
    # Create chunks for workers
    # Extract part of the sample for the current worker
    chunk_size = n_iter//n_workers
    start = i_worker*chunk_size
    if i_worker != n_workers-1:
        end = (i_worker+1)*chunk_size
    else:
        end = n_iter
        
    parameters_sto_chunk = [(p[0], p[1], p[2][start:end]) for p in parameters_sto]
    
    results = run_mc(parameters_sto_chunk, electricity_prod, methods, end-start)
    
    return results

In [ ]:
%%time
task_per_worker(8, 7, option, n_iter, seed=seed)

In [ ]:
task_per_worker = dask.delayed(task_per_worker)

In [ ]:
model_evals = []
for i in range(n_workers):
    model_eval = task_per_worker(n_workers, i, option, n_iter)
    model_evals.append(model_eval)

In [ ]:
%%time
res_intermediate = dask.compute(model_evals)

In [ ]:
res_intermediate = np.array(res_intermediate).squeeze()

In [ ]:
res_all = {}
for m in methods:
    res_array = np.array([])
    for res in res_intermediate:
        res_array = np.hstack([res_array, res[m[-1]]])
    res_all[m[-1]] = res_array

In [ ]:
res_all['land use']

In [ ]:
# Don't forget to close dask client and cluster
client.close()
cluster.close()

In [ ]:
# TODO Save results